In [1]:
import openmc
import numpy as np
from matplotlib import pyplot as plt


# Homework 3: OpenMC Source Distributions and Tallies

## OpenMC Sources

In this homework assignment you will generate source definitions for two different OpenMC geometries. Submit your Jupyter notebook named as "\<last_name>_homework3.ipynb".

1. In this problem, you will model an electron source such as would be used in a medical electron accelerator to create a high-energy bremsstrahlung spectrum. **OpenMC can only transport neutral particles (photons and neutrons) currently, so use a photon source instead**.

  Define a photon source that is Gaussian in both the x and y dimensions with a mean position at x=0 and y=0 
  
  <i>Hint: See the [`openmc.stats.CartesianIndependent`](https://docs.openmc.org/en/stable/pythonapi/generated/openmc.stats.CartesianIndependent.html#openmc.stats.CartesianIndependent) for defining independent spatial distributions and the [`openmc.stats.Normal`](https://docs.openmc.org/en/stable/pythonapi/generated/openmc.stats.Normal.html?highlight=Normal%20distribution) distribution for position.</i>

  The source is positioned 5 cm from the target along the z axis.  The standard deviation of the electron beam is 0.1 cm in the x dimension and 0.5 cm in the y dimension.  The energy of the electrons is 6 MeV.  Define the source so that the electron beam is monodirectional, traveling along the negative z axis.  Model a disk of tungsten (2 cm diameter, 1 mm thick) with its circular face perpendicular to the beam at z=0. All particles are sourced at z=0.

  Do the following:
    - Use the `sample_source` and `plot_geom_and_source` functions below to plot the geometry and source. Examination of the sites returned from the first function can be useful to verify your source definition.
    - Create an `openmc.Model` object, add settings as appropriate, and run the model.


2. Create a simple geometry that resembles a cylindrical brachytherapy seed according to the drawing below.  The spheres are stacked directly on top of one another with sphere 3 being centered in the seed.  Create a uniform spherical volume source in each sphere. Specify the source distributions according to the following table:

![](https://canvas.wisc.edu/files/38445969/download)

**OpenMC can only transport neutral particles (photons and neutrons) currently. Use photon sources for this problem**.

   | Sphere | Source distribution | 
   |------------|-----------------|
   | 1 | The source energy is 30 keV. |
   | 2 | The probability of particle emission from this sphere is 2 times that of sphere 1.  The source energy is Gaussian with a mean of 10 keV and a standard deviation of 2 keV. | 
   | 3	| The probability of particle emission from this sphere is 3 times that of sphere 1.  The source emits photons of 10 keV, 15 keV, and 20 keV with equal probability. |
   | 4	| The probability of particle emission from this sphere is 4 times that of sphere 1. The source emits photons between energies of 20 keV and 30 keV with equal probability. |
   | 5	| The probability of particle emission from this sphere is 5 times that of sphere 1.  The source energy distribution is piecewise linear with relative intensities of 0 at 0 keV, 0.5 at 10 keV, 0 at 20 keV, and 1 at 25 keV. (see [`openmc.stats.Tabular`](https://docs.openmc.org/en/stable/pythonapi/generated/openmc.stats.Tabular.html#openmc-stats-tabular))  |


   Once your model is complete, do the following:

   - Run the model to ensure the source and geometry is valid.
   - Use the `plot_sites` function to annotate a geometry plot with the source sites. 
   
   _Note: the `.plot` methods in OpenMC return a Matplotlib axes you can pass to the `plot_sites` function._

![](geom.png)

**Please See the section below for the part on OpenMC Tallies**


In [2]:
import openmc.lib

from tempfile import TemporaryDirectory
def sample_source(model, n_samples=1000, seed=1):
    """ Initializes the OpenMC problem and samples source sites

    Parameters
    ----------
    model : openmc.Model
        The model object used for the source. Assumed to be a fixed source problem.

    n_samples: Integral
        The number of source sites to sample. Default is 1000.

    seed: Integral
        The random number seed used in OpenMC. Default is 1.
    """
    with TemporaryDirectory() as d:
        model.export_to_model_xml()
        try:
            openmc.lib.init()
            openmc.lib.settings.seed = seed
            openmc.lib.simulation_init()
            sites = openmc.lib.sample_external_source(1000)
        finally:
            openmc.lib.finalize()

    return sites

In [3]:
def plot_sites(ax, sites, basis='xy', arrows=False):
    """Add sites to the matplotlib axes

    Parameters
    ----------
    ax : matplotlib.axes object
        Generally, the axes returned from `openmc.Universe.plot`

    sites : Iterable of openmc.SourceParticle
        The source sites to plot

    basis : One of ('xy, 'yz', 'xz')
        The basis of the plot. Default is 'xy'.

    arrows : bool
        Whether or not to plot the sites as arrows or dots.
    """

    basis_indices = {'xy': [0, 1],
                     'yz': [1, 2],
                     'xz': [0, 2]}

    indices = basis_indices[basis]

    for site in sites:
        x, y = np.asarray(site.r)[indices]
        u, v = np.asarray(site.u)[indices]
        if arrows:
            ax.arrow(x, y, u, v, head_width=0.1)
        else:
            ax.plot(x, y,  marker='o', markerfacecolor='blue')


In [4]:

def plot_geom_and_source(model, source_sites, cell_colors=None):
    """Plots the geometry and source sites.

    Two plots are produced:

        1. A plot in the Y-Z plane with source sites plotted as arrows

        2. A plot in the X-Y plane with source sites plotted as dots to show the spatial distributions.
    """
    for basis in ('yz', 'xy'):
        ax = model.geometry.root_universe.plot(basis=basis,
                                        pixels=(600, 600),
                                        colors=cell_colors,
                                        legend=cell_colors is not None)

        plot_sites(ax, source_sites, basis=basis, arrows=basis == 'yz')

        plt.show()

## OpenMC Tallies

For this section, use the model in the cell below.

Consider the geometry/model in the template. For this problem, assume a source strength of $10^{12}$ neutrons/s. 

Add tallies and modifiers to the input file to accomplish the following tasks.

1. Establish an energy grid for all tallies using an `EnergyFilter` with 44 bins spaced logarithmically between 1e-10 and 10 MeV and 4 bins spaced linearly between 10 and 20 MeV.

2. Determine the energy spectrum of the current leaving the Be sphere and compare to the energy spectrum of the current reentering the Be sphere. Present the results in a `pandas` dataframe.

3. Determine what fraction of the flux that reaches the detector has streamed directly from the source. Plot the flux spectrum that streamed directly in comparison with the total flux spectrum. Use the `CollisionFilter` to accomplish this.

4. Determine the total heating in the NaI detector and what fraction comes from neutrons vs. photons. Report your answers in Watts (W). Use a tally trigger to ensure that the total heating tally has a relative error of less than 5%.

5. Determine the rate of (n,2n) reactions occurring in the Be sphere. Report your answer in reactions/s.

6. Create a mesh tally in the entire water block. For the region that includes the Be sphere, use a 1 cm mesh. For the next 15 cm, use a 3 cm mesh. For the remainder of the mesh, use a 5 cm mesh. Tally both the photon and neutron fluxes. Produce a plot of these mesh tallies in the Y-Z plane.

In [5]:
model = openmc.Model()
model.settings.run_mode = 'fixed source'
model.settings.particles = 500_000
model.settings.batches = 2


In [6]:
### Materials ###
be = openmc.Material(name='Be')
be.set_density('g/cm3', 1.85)
be.add_nuclide('Be9', 1.0)

borated_water = openmc.Material(name='Borated Water')
borated_water.set_density('g/cm3', 1.0)
borated_water.add_nuclide('H1', 2.0)
borated_water.add_nuclide('O16', 1.0)
borated_water.add_nuclide('B10', 0.05)

tritiated_titanium = openmc.Material(name='Tritiated Titanium')
tritiated_titanium.set_density('g/cm3', 4.5)
tritiated_titanium.add_nuclide('Ti48', 0.95)
tritiated_titanium.add_nuclide('H3', 0.05)

sodium_iodide = openmc.Material(name='Sodium Iodide')
sodium_iodide.set_density('g/cm3', 3.67)
sodium_iodide.add_nuclide('Na23', 1.0)
sodium_iodide.add_nuclide('I127', 1.0)

model.materials = openmc.Materials([be, borated_water, tritiated_titanium, sodium_iodide])

In [55]:
### Surfaces ###
sphere = openmc.Sphere(r=25)

rcc1 = openmc.model.RightCircularCylinder((0, -50.5, 0), height=51, axis='y', radius=2.0)

rpp = openmc.model.RectangularParallelepiped(-50, 50, -50, 50, -50, 50, boundary_type='vacuum')

rcc2 = openmc.model.RightCircularCylinder((0, 0, 0), height=100, axis='y', radius=4.0)
rcc2 = rcc2.rotate((0, 0, 45))

rcc3 = openmc.model.RightCircularCylinder((0, 35, 0), height=7.0, axis='y', radius=3.9)
rcc3 = rcc3.rotate((0, 0, 45))

y_min = openmc.YPlane(y0=-0.5)
y_max = openmc.YPlane(y0=0.0)

In [59]:
### Cells ###
be_sphere = openmc.Cell(fill=be, region=-sphere & + rcc1)

target_region = -rcc1 & +y_min
target = openmc.Cell(fill=tritiated_titanium, region=target_region)

beam_tube = openmc.Cell(region=-rcc1 & -y_min & -rpp)

water_jacket = openmc.Cell(fill=borated_water, region=+sphere & -rpp & +rcc1 & +rcc2)

detector_tube = openmc.Cell(region=+sphere & -rpp & -rcc2 & +rcc3)

detector = openmc.Cell(fill=sodium_iodide, region=-rcc3)

model.geometry = openmc.Geometry([be_sphere, target, beam_tube, water_jacket, detector_tube, detector])